# Data exploration

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install geopandas rtree &> /dev/null # install necessary packages 

import geopandas as gpd
import pandas as pd
import numpy as np
import rtree
import os
import pickle
import matplotlib.pyplot as plt

print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/shared/ssc22-case-comp/dataset


In [ ]:
import numpy as np

def w_avg(df, name_col, val_cols, w_col='tests'):
    """ Computes weighted averages of specified columns"""
    '''
    df: target data frame.
    name_col: String. The grouping column.
    val_cols: list of strings. Names of the columns to average over.
    w_col: String. Name of the column that is used for weighting the averages.
    '''
    
    names = df[name_col].unique() 
    col_list = [name_col, val_cols[0], val_cols[1], val_cols[2], w_col, 'devices', 'DA_POP']
    result_df = pd.DataFrame(columns = col_list)    
    
    for name in names: # for each ID
        # extract a group
        temp_table = df[df[name_col]==name] 
        # compute the weightes averages
        temp_d_avg = np.average(temp_table[val_cols[0]], weights=temp_table[w_col])
        temp_u_avg = np.average(temp_table[val_cols[1]], weights=temp_table[w_col])
        temp_l_avg = np.average(temp_table[val_cols[2]], weights=temp_table[w_col])
        temp_w = np.sum(temp_table[w_col])
        temp_sactype = np.average(temp_table['SACTYPE'], weights = temp_table[w_col])
        # temp_da_pop = np.sum(temp_table['DA_POP']) ## apply weighted sum?
        temp_devices = np.sum(temp_table['devices'])
        
        # add the averages to the result dataframe
        result_df = result_df.append(dict(zip(col_list, [name, temp_d_avg, 
                                                         temp_u_avg, 
                                                         temp_l_avg, 
                                                         temp_w,
                                                         temp_devices,
                                                         temp_sactype
                                                        #  temp_da_pop
                                                         ])), ignore_index=True)
    return result_df

In [43]:
a_df = pd.DataFrame(columns = ['time', 'value'])
t = ['2019-Q1', '2019-Q2']
v = [1,2]

aa = pd.DataFrame(data = zip(t, v), columns = ['time', 'value'])
aa

a_dict = {'2019-Q1': '20191', '2019-Q2':'20192'}
aa['new_col'] = 0

for i in range(len(aa)):
  aa['new_col'][i] = a_dict[aa['time'][i]]
aa

aa['new_col'] = aa['time'].replace(to_replace=['2019-Q1', '2019-Q2'], value=['1','2'])

t_vals = aa['time'].unique().sort()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,time,value,new_col
0,2019-Q1,1,20191
1,2019-Q2,2,20192


In [58]:
t_vals = sorted(aa['time'].unique())  # sorts year-quarter values

t_ints = [*range(1,len(t_vals)+1)] # creates a list of integers [1, 2, ...]

aa['new_col'] = aa['time'].replace(to_replace=t_vals, value=t_ints)

aa

['2019-Q1', '2019-Q2']


,time,value,new_col
0,2019-Q1,1,1
1,2019-Q2,2,2


In [ ]:
a_dict = {'2019-Q1': '20191', '2019-Q2':'20192'}
aa['new_col'] = 0

for i in range(len(aa)):
  aa['new_col'][i] = a_dict[aa['time'][i]]
aa

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,time,value,new_col
0,2019-Q1,1,20191
1,2019-Q2,2,20192


In [ ]:
# Split, compute the weighted averages for the speeds, merge with the boundary data, convert the time columns, and rbind.

# can_df = {}

years = ['2019', '2020', '2021']
quarters = ['Q1', 'Q2', 'Q3', 'Q4']
conn_types = ['fixed', 'mobile']

yqs = []
for y in years:
    for q in quarters:
        yqs.append(y + '-' + q)
# yqs
province_names = ['Northwest Territories', 'Yukon', 'British Columbia', 
                  'Alberta', 'Saskatchewan', 'Manitoba', 'Ontario', 
                  'Newfoundland and Labrador', 'Quebec', 'New Brunswick', 
                  'Nova Scotia', 'Prince Edward Island', 'Nunavut']

muni = gpd.read_file('./boundaries/statcan-census-subdivision/')

for i,p in enumerate(province_names):
    p_df = pd.DataFrame(data=[], columns=['CSDUID', 'PRUID', 'CDUID', 'CSDTYPE', 'geometry', 'avg_d_mbps',
       'avg_u_mbps', 'avg_lat_ms', 'tests', 'devices', 'DA_POP', 'time',
       'conn_type'])
    
    for y in yqs:
        for c in conn_types:            
            
            print('{p}, {yq}, {c}'.format(yq = y, c = c, p = p))

            ## load the quarterly data
            try:
                file_path = './' + province_names[i] + '/by_year_quarter/' + province_names[i]+'-'+ y + '.shp'
                p_file_load = gpd.read_file(file_path)
                p_file_load = p_file_load[p_file_load['conn_type']==c]

                ## change the download and upload speed's unit to mbps
                p_file_load['avg_d_mbps'] = p_file_load['avg_d_kbps'] / 1000
                p_file_load['avg_u_mbps'] = p_file_load['avg_u_kbps'] / 1000

                ## make 'time' column by merging year and quarter, then convert it to the datetime format.
                # months = ['03-31', '06-30', '09-30', '12-31']
                months = ['1', '2', '3', '4']
                p_file_load['month'] = p_file_load['quarter'].replace(quarters, months)
                p_file_load['time'] = p_file_load['year'] + '-' + p_file_load['month']
                p_file_load['time'] = pd.to_datetime(p_file_load['time'])

                ## drop unused columns
                p_file_load = p_file_load.drop(columns = ['year','month','quarter','avg_d_kbps', 'avg_u_kbps', 'DAUID'])

                ## filter the boundary data by the province
                pruid = str(p_file_load['PRUID'].unique()[0])
                b_file_load = muni[muni['PRUID']==pruid]
            
            except:
                break
            
            ## match the crs
            p_file = p_file_load.to_crs(4326)
            b_file = b_file_load.to_crs(4326)

            ## perform .sjoin to label each tile with the CSDUID
            temp_merged = gpd.sjoin(p_file, b_file, how='inner', predicate='intersects')
            temp_merged = temp_merged.drop(columns=['geometry']).drop_duplicates(subset='quadkey', keep='first')

            ## compute the weighted average speeds
            test_w_avg = w_avg(temp_merged, 'CSDUID', ['avg_d_mbps', 'avg_u_mbps', 'avg_lat_ms'])

            ## Merge other features on the w_avg table.
            ## The final table will have:
            ## ['CSDUID', 'avg_d_mbps', 'avg_u_mbps', 'avg_lat_ms', 'tests', 'devices', 'DA_POP'] ## from w_avg table
            ## ['CSDUID', 'time', 'conn_type'] ## from temp_merged. 
            ## ['CSDUID', 'PRUID', 'CDUID', 'CSDTYPE', 'geometry'] ## from b_file

            m1 = temp_merged[['time', 'CSDUID','conn_type']].drop_duplicates() ## 
            m2 = b_file[['CSDUID', 'PRUID', 'CDUID', 'CSDTYPE', 'geometry']] ## geopandas boundary file

            final1 = m2.merge(test_w_avg, how='inner', on='CSDUID')
            final2 = final1.merge(m1, how='inner', on='CSDUID')

            p_df = p_df.append(final2)
    can_df[p] = p_df
    with open('./canada-avg-by-csd.p', 'wb') as f:
        pickle.dump(can_df, f)

New Brunswick, 2019-Q1, fixed
New Brunswick, 2019-Q1, mobile


/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


New Brunswick, 2019-Q2, fixed
New Brunswick, 2019-Q2, mobile
New Brunswick, 2019-Q3, fixed
New Brunswick, 2019-Q3, mobile
New Brunswick, 2019-Q4, fixed
New Brunswick, 2019-Q4, mobile
New Brunswick, 2020-Q1, fixed
New Brunswick, 2020-Q1, mobile
New Brunswick, 2020-Q2, fixed
New Brunswick, 2020-Q2, mobile
New Brunswick, 2020-Q3, fixed
New Brunswick, 2020-Q3, mobile
New Brunswick, 2020-Q4, fixed
New Brunswick, 2020-Q4, mobile
New Brunswick, 2021-Q1, fixed
New Brunswick, 2021-Q1, mobile
New Brunswick, 2021-Q2, fixed
New Brunswick, 2021-Q2, mobile
New Brunswick, 2021-Q3, fixed
New Brunswick, 2021-Q3, mobile
New Brunswick, 2021-Q4, fixed
New Brunswick, 2021-Q4, mobile
Nova Scotia, 2019-Q1, fixed
Nova Scotia, 2019-Q1, mobile
Nova Scotia, 2019-Q2, fixed
Nova Scotia, 2019-Q2, mobile
Nova Scotia, 2019-Q3, fixed
Nova Scotia, 2019-Q3, mobile
Nova Scotia, 2019-Q4, fixed
Nova Scotia, 2019-Q4, mobile
Nova Scotia, 2020-Q1, fixed
Nova Scotia, 2020-Q1, mobile
Nova Scotia, 2020-Q2, fixed
Nova Scotia, 202

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Nunavut, 2021-Q4, mobile


,CSDUID,PRUID,CDUID,CSDTYPE,geometry,avg_d_mbps,avg_u_mbps,avg_lat_ms,tests,devices,DA_POP,time,conn_type
0,6204003,62,6204,CY,"POLYGON ((-68.56990 63.71436, -68.60673 63.713...",39.944,45.873000,56.000000,2,1,NaN,2019-09-30,fixed
0,6204030,62,6204,NO,"POLYGON ((-74.54867 76.28105, -74.30043 76.000...",53.060,9.693333,57.333333,3,3,NaN,2019-12-31,fixed
0,6204030,62,6204,NO,"POLYGON ((-74.54867 76.28105, -74.30043 76.000...",32.034,5.953000,59.000000,1,1,NaN,2020-03-31,fixed
1,6205023,62,6205,HAM,"POLYGON ((-96.01267 64.37267, -96.00079 64.371...",20.025,6.900000,30.000000,1,1,NaN,2020-03-31,fixed
0,6208098,62,6208,NO,"POLYGON ((-95.07684 72.45856, -94.70974 72.397...",3.920,9.202000,5.000000,1,1,NaN,2020-06-30,fixed


In [ ]:
## check duplicates

print(len(test_merge['quadkey']))
print(len(test_merge['quadkey'].unique()))
aa = test_merge[test_merge.duplicated(subset=['quadkey'])==True]
quadkeys = test_merge['quadkey']

test_merge[quadkeys.isin(quadkeys[quadkeys.duplicated()])].sort_values(by='quadkey')

In [ ]:
province = 'New Brunswick'
read_path = './' + province + '/' + province +'-speed-tiles.shp'
nb = gpd.read_file(read_path)

years = ['2019', '2020', '2021']
quarters = ['Q1', 'Q2', 'Q3', 'Q4']

for year in years:
    for quarter in quarters:
        
        nb_index = (nb['year']==year) & (nb['quarter']==quarter)

        out_data = nb[nb_index]
        
        time_name = year + '-' + quarter + '.shp'
        out_path = './' + province + "/by_year_quarter/" + province + '-' + time_name
        try:
            out_data.to_file(out_path)
        except:
            break

In [ ]:
os.getcwd()

'/content'

In [ ]:
os.chdir('/content/drive/MyDrive/shared/ssc22-case-comp/dataset/')
with open('./canada-avg-by-csd.p', 'rb') as f:
  data = pickle.load(f)

AttributeError: ignored

In [ ]:
combined_df = pd.DataFrame(columns = data['Northwest Territories'].columns)
for k in data.keys():
  combined_df = combined_df.append(data[k])

In [ ]:
combined_df.head()

,CSDUID,PRUID,CDUID,CSDTYPE,geometry,avg_d_mbps,avg_u_mbps,avg_lat_ms,tests,devices,DA_POP,time,conn_type
0,6101017,61,6101,T,"POLYGON ((-133.48192 68.32845, -133.44690 68.3...",14.831457,6.289283,36.847826,46,16,NaN,2019-03-31,fixed
1,6101025,61,6101,HAM,"POLYGON ((-134.99221 68.22099, -134.99302 68.2...",13.439500,1.481000,26.000000,2,2,NaN,2019-03-31,fixed
2,6101036,61,6101,HAM,"POLYGON ((-132.94668 69.40747, -132.94669 69.3...",5.038000,1.317000,54.000000,1,1,NaN,2019-03-31,fixed
3,6102005,61,6102,HAM,"POLYGON ((-125.49035 64.95092, -125.46781 64.9...",13.896667,1.281667,25.666667,3,2,NaN,2019-03-31,fixed
4,6102007,61,6102,T,"POLYGON ((-126.62243 65.27435, -126.62321 65.2...",28.425000,3.389514,28.783784,37,13,NaN,2019-03-31,fixed


In [ ]:
combined_df.to_csv('./canada-avg-by-csd.csv')

In [ ]:
with open('./canada-avg-by-csd-combined.p', 'wb') as file:
  pickle.dump(combined_df, file)

In [ ]:
gdf = gpd.GeoDataFrame(combined_df, geometry='geometry')

In [ ]:
gdf.to_file('./canada-avg-by-csd.shp')

DriverSupportError: ignored